In [1]:
import pandas as pd
import csv

In [2]:
test_df = pd.read_csv("./data/test.csv")

In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      34 non-null     object
 1   이름      34 non-null     object
 2   지역      34 non-null     object
 3   날짜      34 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [5]:
df = test_df.sort_values('날짜')

# 중복 제거

In [6]:
df = df.drop_duplicates()

# ID, 이름에 에러가 있는 데이터 제거

In [7]:
id_filter = df["ID"].str.len() > 10
df = df[id_filter]

In [8]:
name_filter = df["이름"].str.contains("_T4")
df = df[name_filter]

# 매칭 취소자 제거

In [9]:
cancel_df = pd.read_csv("./data/cancel.csv")

In [10]:
cancel_filter = df[df['이름'].isin(set(cancel_df['이름'].unique()))].index

In [11]:
df = df.drop(cancel_filter)

# 간단 매칭 알고리즘

In [23]:
for (k1, k2), group in df.groupby(['날짜', '지역']):
    if len(group) > 1:
        print((k1, k2))
        display(group)

(20221229, '서울-강남')


,ID,이름,지역,날짜
14,U041ERYHTV3,테스트1_T4001,서울-강남,20221229
26,U041ERYHTV3,테스트3_T4012,서울-강남,20221229
19,U041ERYHTV3,테스트2_T4002,서울-강남,20221229


(20221231, '서울-강남')


,ID,이름,지역,날짜
29,U041ERYHTV3,테스트3_T4012,서울-강남,20221231
23,U041ERYHTV3,테스트2_T4002,서울-강남,20221231
16,U041ERYHTV3,테스트1_T4001,서울-강남,20221231
7,U041ERYHTV3,김부캠_T4000,서울-강남,20221231


(20221231, '서울-강북')


,ID,이름,지역,날짜
30,U041ERYHTV3,테스트3_T4012,서울-강북,20221231
6,U041ERYHTV3,김부캠_T4000,서울-강북,20221231


(20230101, '서울-강남')


,ID,이름,지역,날짜
21,U041ERYHTV3,테스트2_T4002,서울-강남,20230101
32,U041ERYHTV3,테스트3_T4012,서울-강남,20230101
18,U041ERYHTV3,테스트1_T4001,서울-강남,20230101


In [12]:
# 그냥 시간 순서대로 매칭~
visited = []
result = []
for (k1, k2), group in df.groupby(['날짜', '지역']):
    _group = group[~group['이름'].isin(visited)]
    if len(_group) > 1:
        print((k1, k2))
        display(_group)
        for n, i in zip(_group['이름'].values, _group['ID'].values):
            visited.append(n)
            result.append([i, n, k2, k1])
print(f"전체 매칭 신청 인원 수: {df['이름'].nunique()} \n매칭 완료 인원 수: {len(result)}")

(20221229, '서울-강남')


,ID,이름,지역,날짜
14,U041ERYHTV3,테스트1_T4001,서울-강남,20221229
26,U041ERYHTV3,테스트3_T4012,서울-강남,20221229
19,U041ERYHTV3,테스트2_T4002,서울-강남,20221229


전체 매칭 신청 인원 수: 4 
매칭 완료 인원 수: 3


In [13]:
with open('./data/result.csv', 'w', newline='') as csvfile:
    wr = csv.writer(csvfile)
    wr.writerow(['ID', '이름', '지역', '날짜'])   # 열 제목

In [14]:
with open('./data/result.csv', 'a', newline='') as csvfile:
        ar = csv.writer(csvfile)
        for r in result:
            ar.writerow(r)